### I will create a visualization for tracking Donald Trump’s approval ratings throughout his presidency.

In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [2]:
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [3]:
topline_df = pd.read_csv('assets/approval_topline.csv')
polls_df = pd.read_csv('assets/approval_poll_list.csv')

In [4]:
topline_df.head()

,president,subgroup,modeldate,approve_estimate,approve_hi,approve_lo,disapprove_estimate,disapprove_hi,disapprove_lo,timestamp
0,Donald Trump,Voters,4/15/2020,44.761500,48.405868,41.117133,51.533103,55.280090,47.786116,10:03:57 15 Apr 2020
1,Donald Trump,Adults,4/15/2020,43.994962,47.722998,40.266926,50.842083,55.956679,45.727487,10:02:28 15 Apr 2020
2,Donald Trump,All polls,4/15/2020,44.356560,48.075278,40.637841,51.377554,55.887691,46.867418,10:01:29 15 Apr 2020
3,Donald Trump,Adults,4/14/2020,43.764737,47.704991,39.824482,50.748517,56.060162,45.436873,19:23:26 14 Apr 2020
4,Donald Trump,Voters,4/14/2020,44.645348,48.374384,40.916311,51.774275,55.606283,47.942267,19:24:55 14 Apr 2020


In [5]:
polls_df.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,4/9/2020,1/20/2017,1/22/2017,Morning Consult,B/C,1992.0,rv,0.696984,...,37.0,44.822311,38.224809,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,14:28:27 9 Apr 2020
1,Donald Trump,All polls,4/9/2020,1/20/2017,1/22/2017,Gallup,B,1500.0,a,0.241682,...,45.0,45.713854,43.635224,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,14:28:27 9 Apr 2020
2,Donald Trump,All polls,4/9/2020,1/21/2017,1/23/2017,Gallup,B,1500.0,a,0.223234,...,46.0,45.713854,44.635224,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49262,77274,1/24/2017,14:28:27 9 Apr 2020
3,Donald Trump,All polls,4/9/2020,1/20/2017,1/24/2017,Ipsos,B-,1632.0,a,0.150785,...,45.2,42.961893,43.986429,NaN,T,http://polling.reuters.com/#poll/CP3_2/,49426,77599,3/1/2017,14:28:27 9 Apr 2020
4,Donald Trump,All polls,4/9/2020,1/20/2017,1/25/2017,Quinnipiac University,B+,1190.0,rv,1.494494,...,44.0,37.586166,42.839882,NaN,NaN,https://poll.qu.edu/national/release-detail?Re...,49260,77272,1/26/2017,14:28:27 9 Apr 2020


In [6]:
polls_df.shape

(11556, 22)

In [7]:
topline_df['dates'] = pd.DatetimeIndex(topline_df['timestamp']).strftime('%Y-%m-%d')
topline_df = topline_df[topline_df['subgroup'] == 'Adults']
topline_df.head()

,president,subgroup,modeldate,approve_estimate,approve_hi,approve_lo,disapprove_estimate,disapprove_hi,disapprove_lo,timestamp,dates
1,Donald Trump,Adults,4/15/2020,43.994962,47.722998,40.266926,50.842083,55.956679,45.727487,10:02:28 15 Apr 2020,2020-04-15
3,Donald Trump,Adults,4/14/2020,43.764737,47.704991,39.824482,50.748517,56.060162,45.436873,19:23:26 14 Apr 2020,2020-04-14
7,Donald Trump,Adults,4/13/2020,43.651510,47.588870,39.714150,50.741525,56.043112,45.439937,19:15:25 13 Apr 2020,2020-04-13
10,Donald Trump,Adults,4/12/2020,43.737121,47.754248,39.719995,50.141861,55.254046,45.029675,20:19:23 12 Apr 2020,2020-04-12
13,Donald Trump,Adults,4/11/2020,43.753236,47.756355,39.750117,50.112176,55.208027,45.016326,19:42:24 11 Apr 2020,2020-04-11


In [8]:
new_df = pd.DataFrame(
    {
        'subgroup' : topline_df['subgroup'],
        'dates' : topline_df['dates'],
        'disapprove' : topline_df['disapprove_estimate'],
        'approve' : topline_df['approve_estimate']
    }
)

In [10]:

dates = pd.DatetimeIndex(polls_df['timestamp']).strftime('%Y-%m-%d')

adult_poll = pd.DataFrame(
    {
        'subgroup' : polls_df['subgroup'],
        'dates' : dates,
        'disapprove' : polls_df['disapprove'],
        'approve' : polls_df['approve']
    }
)

adult_poll = adult_poll[adult_poll['subgroup'] == 'Adults']

adult_poll = pd.concat([adult_poll, new_df])

disap_chart = alt.Chart(adult_poll).mark_point(color = 'red').encode(
    x="dates",
    y="disapprove"
)

app_chart = alt.Chart(adult_poll).mark_point(color = 'green').encode(
    x="dates",
    y="approve"
)

disap_chart + app_chart

alt.LayerChart(...)

In [11]:
disp_uncert = alt.Chart(adult_poll).mark_boxplot(extent='min-max').encode(
    x='dates:O',
    y='disapprove:Q'
)

app_uncert = alt.Chart(adult_poll).mark_boxplot(extent='min-max').encode(
    x='dates:O',
    y='approve:Q'
)

disp_uncert + app_uncert

alt.LayerChart(...)